In [ ]:
!pip install torch-geometric

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import requests

2023-06-01 14:49:28.900871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 14:49:29.219194: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-01 14:49:29.219215: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-01 14:49:30.157052: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

### Collect dataset

In [29]:
url = 'http://api.scraperlink.com/investpy/?email=your@email.com&type=historical_data&product=cryptos&symbol=BTC&from_date=10/01/2013&to_date=2/1/2023'
response = requests.request('GET', url)
prices = response.json()['data']
data = pd.DataFrame(prices)

In [30]:
data.head()

,direction_color,rowDate,rowDateRaw,rowDateTimestamp,last_close,last_open,last_max,last_min,volume,volumeRaw,change_precent,last_closeRaw,last_openRaw,last_maxRaw,last_minRaw,change_precentRaw
0,greenFont,"Feb 01, 2023",1675209600,2023-02-01T00:00:00Z,"23,725.6","23,124.7","23,784.5","22,809.6",333.18K,333182,2.60,23725.58007812500000,23124.74023437500000,23784.51171875000000,22809.64843750000000,2.596756
1,greenFont,"Jan 31, 2023",1675123200,2023-01-31T00:00:00Z,"23,125.1","22,832.2","23,262.9","22,724.9",289.38K,289377,1.28,23125.07812500000000,22832.18750000000000,23262.93164062500000,22724.92187500000000,1.282797
2,redFont,"Jan 30, 2023",1675036800,2023-01-30T00:00:00Z,"22,832.2","23,746.0","23,794.0","22,560.0",339.62K,339618,-3.88,22832.18750000000000,23745.96289062500000,23793.99804687500000,22560.01171875000000,-3.877011
3,greenFont,"Jan 29, 2023",1674950400,2023-01-29T00:00:00Z,"23,753.1","23,023.5","23,952.9","22,973.3",320.86K,320863,3.15,23753.09765625000000,23023.52343750000000,23952.85937500000000,22973.33593750000000,3.149343
4,redFont,"Jan 28, 2023",1674864000,2023-01-28T00:00:00Z,"23,027.9","23,074.6","23,182.3","22,889.8",156.96K,156957,-0.20,23027.87109375000000,23074.59570312500000,23182.31054687500000,22889.75976562500000,-0.202494


In [31]:
prices = data[['last_close', 'last_open', 'last_max', 'last_min']].replace({',':''}, regex=True).astype(np.float64)
prices

,last_close,last_open,last_max,last_min
0,23725.6,23124.7,23784.5,22809.6
1,23125.1,22832.2,23262.9,22724.9
2,22832.2,23746.0,23794.0,22560.0
3,23753.1,23023.5,23952.9,22973.3
4,23027.9,23074.6,23182.3,22889.8
...,...,...,...,...
3406,136.7,136.8,138.0,135.3
3407,136.8,131.0,139.8,128.5
3408,131.0,123.0,132.0,120.1
3409,123.0,140.3,141.9,109.7


In [32]:
predicted_prices = prices['last_close'][1:]
data = prices.copy()[:-1]
data['target'] = np.int32( np.array(predicted_prices)>=data['last_close'] ) #whether price will go up or down
data

,last_close,last_open,last_max,last_min,target
0,23725.6,23124.7,23784.5,22809.6,0
1,23125.1,22832.2,23262.9,22724.9,0
2,22832.2,23746.0,23794.0,22560.0,1
3,23753.1,23023.5,23952.9,22973.3,0
4,23027.9,23074.6,23182.3,22889.8,1
...,...,...,...,...,...
3405,137.8,136.7,138.0,134.1,0
3406,136.7,136.8,138.0,135.3,1
3407,136.8,131.0,139.8,128.5,0
3408,131.0,123.0,132.0,120.1,0


In [57]:
import gudhi

class predictor:
    def __init__(self, data, k = 20, max_dim=2):
        X=[]
        self.max_dim = max_dim
        for i in range(k-1, len(data)):
            x = np.array( data.iloc[i-k+1:i+1, :4] )
            x -= np.min(x)
            x /= np.max(x)
            tmp = np.zeros((k,5))
            tmp[:,:-1] = x
            tmp[:,-1] = list(range(k))
            X.append( tmp )

        self.X = np.array(X)
        print(self.X)

    def persistent_analysis(self, data):
        rips_complex = gudhi.RipsComplex(points=data)
        simplex_tree = rips_complex.create_simplex_tree(max_dimension=self.max_dim)
        diag = simplex_tree.compute_persistence()
        simplices = list(simplex_tree.get_simplices())
        simplices.sort(key=lambda x: x[1])
        simplices = simplices[len(data):]
        return simplices

        
        

model = predictor(data, k=10, max_dim=2)

[[[0.85673768 0.47800328 0.89386109 0.2794025  0.        ]
  [0.47825539 0.29364679 0.56510778 0.2260179  1.        ]
  [0.29364679 0.86959536 0.89984873 0.12208496 2.        ]
  ...
  [0.43413589 0.16778016 0.89039455 0.         7.        ]
  [0.16778016 0.34665322 0.49779402 0.09504601 8.        ]
  [0.346149   0.21599647 0.50138661 0.10185302 9.        ]]

 [[0.49211608 0.3124118  0.57666114 0.24657954 0.        ]
  [0.3124118  0.8730597  0.90250936 0.14540769 1.        ]
  [0.87741579 0.42978097 1.         0.39898153 2.        ]
  ...
  [0.18988895 0.36401006 0.51113565 0.11908706 7.        ]
  [0.36351923 0.23682435 0.5146328  0.12571323 8.        ]
  [0.2360881  0.2777471  0.45751273 0.         9.        ]]

 [[0.3124118  0.8730597  0.90250936 0.14540769 0.        ]
  [0.87741579 0.42978097 1.         0.39898153 1.        ]
  [0.43248052 0.46113258 0.52721026 0.3477514  2.        ]
  ...
  [0.36351923 0.23682435 0.5146328  0.12571323 7.        ]
  [0.2360881  0.2777471  0.4575127

In [58]:
model.persistent_analysis(model.X[0])

[([8, 9], 1.0241805852416197),
 ([5, 6], 1.0244367485581543),
 ([4, 5], 1.0345077903465825),
 ([7, 8], 1.1252154756128931),
 ([6, 7], 1.1310104980456301),
 ([0, 1], 1.1349734843731865),
 ([3, 4], 1.203588276678874),
 ([1, 2], 1.2201027216977152),
 ([2, 3], 1.2736351595265714),
 ([4, 5, 6], 2.0006861782426495),
 ([4, 6], 2.0006861782426495),
 ([5, 6, 7], 2.0288560870278345),
 ([5, 7], 2.0288560870278345),
 ([6, 7, 8], 2.0296623195614365),
 ([6, 8], 2.0296623195614365),
 ([7, 8, 9], 2.042490568300785),
 ([7, 9], 2.042490568300785),
 ([3, 4, 5], 2.079934953281566),
 ([3, 5], 2.079934953281566),
 ([1, 2, 3], 2.094003296814179),
 ([1, 3], 2.094003296814179),
 ([2, 3, 4], 2.094310520194997),
 ([2, 4], 2.094310520194997),
 ([0, 1, 2], 2.1201887568402977),
 ([0, 2], 2.1201887568402977),
 ([0, 1, 3], 3.004376761481277),
 ([0, 2, 3], 3.004376761481277),
 ([0, 3], 3.004376761481277),
 ([1, 2, 4], 3.006738680179423),
 ([1, 3, 4], 3.006738680179423),
 ([1, 4], 3.006738680179423),
 ([6, 7, 9], 3.016